In [1]:
from pathlib import Path
import pandas as pd
import os
import csv
import numpy as np
import requests
import time
import io
import yfinance as yf
import warnings
# Initial imports
#%tensorflow_version 1.x
import tensorflow
import mibian

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import lstm
import psutil
#import main_lstm
import main_rf
#import trial_lstm
psutil.virtual_memory
import working_lstm
import time
import robust_lstm
import psutil
values = psutil.virtual_memory()
avail = values.available >> 20
avail

17110

In [2]:

###The first process, for the first features needed for the program
def tp(end):
    last = time.perf_counter()
    #time_last = ('seconds passed since last function:', round((last - end)))
    

    time_total = ('seconds passed since start:', round((end - start)))
    values = psutil.virtual_memory()
    avail = values.available >> 20
    print(f'current time is {last}')
    print(f'Memory available in MBs: {avail}')
    print(f'time since start: {time_total}')
    #print(f'time since last function: {time_last}')
    return last


def feature_engineering(data, symbol):
    data = data[data['Symbol']== symbol]

    data = data.set_index(pd.to_datetime(data.DataDate, infer_datetime_format=True))

    #data = data.drop(columns=['DataDate'])
    data['ExpirationDate'] = data['ExpirationDate'].apply(pd.Timestamp)
    
    data['Date'] = data['DataDate'].apply(pd.Timestamp)
    #data = data.loc[data['PutCall'] == 'put']

    #data['ExpirationDate'] =  pd.to_datetime(data['ExpirationDate'], infer_datetime_format=True)
    #data['DataDate'] =  pd.to_datetime(data['DataDate'], infer_datetime_format=True)
    data['time delta'] = data['ExpirationDate'] - data['Date']
    data['StrikePrice'] = pd.to_numeric(data['StrikePrice'])
    data['UnderlyingPrice'] = pd.to_numeric(data['UnderlyingPrice'])
    data['% out'] = data['StrikePrice'] / data['UnderlyingPrice'] - 1
    data['exp?'] = data['ExpirationDate'] == data['Date']
    #data['ExpirationDate']= data['ExpirationDate'].to_string
    #data['DataDate'] = data['DataDate'].to_string
    data['DateKey'] = data["Date"].astype(str) + data["ExpirationDate"].astype(str) 
    

    #data['xp'] = data.loc[data['exp?'] == True, 'AskPrice']
    #data['xp']=data.OptionKey.map(xp.set_index('OptionKey')['xp'])

    return data



###Map the expiration value of each Option to all its rows
def map_exp(df):
    df['optionkey'] = df['optionkey'].str.upper()

    df['optionkey'] = df['optionkey'].map(lambda x:str(x)[:-10])

    xp = df
    xp.loc[xp['exp?']== True, 'xp' ] = xp.loc[df['exp?'] == True, 'AskPrice']
    xp = xp.loc[xp['exp?']== True]
    xp= xp[['optionkey', 'xp']]
    df['xp']=df.optionkey.map(xp.set_index('optionkey')['xp'])
    df['profit'] = df['AskPrice'].astype(float) - df['xp'].astype(float)

    return df

###Pass the data frame with features in to this
###This will create values for the option spreads we will later buy or sell
def create_spreads(data, direction):

    data['DataDate'] = data.index
    data['DateKey']=data['DataDate'].astype('str')+data['ExpirationDate'].astype('str') 
    
    data['DataDate'] = data['DataDate'].apply(pd.Timestamp)
    data.groupby('DateKey') 
    data.sort_values('DataDate') 
    data['Prev_Strike']=data['StrikePrice'].shift(direction)
    data['Prev_AskPrice']=data['AskPrice'].shift(direction)
    data['Prev_DateKey']=data['DateKey'].shift(direction)
    data['Prev_xp']=data['xp'].shift(direction)
      #Create nan values for the crossover days
  
    data['spread_premium'] = data['AskPrice'].astype(float) - data['Prev_AskPrice'].astype(float)
    data['spread_xp'] = data['xp'].astype(float) - data['Prev_xp'].astype(float)
    data['spread_profit'] = data['spread_premium'].astype(float) - data['spread_xp'].astype(float)
    data['spread_collateral'] = data['StrikePrice'] - data['Prev_Strike'] - data['spread_premium']
    
    for idx, row in data.iterrows():
        if row['Prev_DateKey']!=row['DateKey']:
            row['Prev_Strike']=np.nan
            row['Prev_AskPrice']=np.nan
            row['Prev_xp']=np.nan
            row['spread_premium'] = np.nan
            row['spread_xp'] = np.nan
            row['spread_profit'] = np.nan
    
        
    return data

###Eliminate all nulls immediately to avoid calculation discrepancies
def create_nulls(df):
     for idx, row in df.iterrows():
        if row['Prev_DateKey']!=row['DateKey']:
            row['Prev_Strike']=np.nan
            row['Prev_AskPrice']=np.nan
            row['Prev_xp']=np.nan
            row['spread_premium'] = np.nan
            row['spread_xp'] = np.nan
            row['spread_profit'] = np.nan
    
        return df   

###Create features and their signals
def get_option_volume(option_df, Symbol):   
    mf=option_df[option_df['Symbol']== Symbol]
    #mf = mf.drop(columns=['DataDate'])

    mf['Volume'] = mf['Volume'].astype(int)
    mf['Total Volume'] = mf.groupby(['Date']).Volume.transform('sum')
    mf['Call Volume'] = mf.loc[mf['PutCall'] == 'call', ['Date', 'Volume']].groupby(['Date']).sum()
    mf['Put Volume'] = mf.loc[mf['PutCall'] == 'put', ['Date', 'Volume']].groupby(['Date']).sum()
    volume = mf.groupby('Date').agg({'Call Volume': 'sum', 'Put Volume' : 'sum'})
    volume['Total Volume'] = volume.loc[:,['Call Volume','Put Volume']].sum(axis=1)

    #Create put signals
    volume['Percent of Put'] = volume['Put Volume']/volume['Total Volume']
    volume['Put signals'] = volume['Percent of Put'] > .6

    #Create Call volume
    volume['Percent of calls'] = volume['Call Volume']/volume['Total Volume']
    volume['Call signals'] = volume['Percent of calls'] > .6
    volume_ratio = 20 #3
    volume['Option rVol'] = volume['Total Volume']/ volume['Total Volume'].rolling(window = volume_ratio).mean()


    volume['option rvol delta'] = volume['Option rVol'].dropna().pct_change()

    cond=[volume['Percent of Put'] > .75, volume['Percent of Put'] > .7, volume['Percent of Put'] > .65, volume['Percent of Put'] > .6,\
      volume['Percent of Put'] < .25, volume['Percent of Put'] < .3, volume['Percent of Put'] < .35, volume['Percent of Put'] < .4]
    values=[-2.5, -2, -1.5, -1, 2.5, 2, 1.5, 1]
    volume['Option Signal']=np.select(cond,values)
    
    return volume

# cond=[df['Total_Summer']>df['Total_Winter'],df['Total_Summer']<df['Total_Winter'],df['Total_Summer'].eq(df['Total_Winter'])]
# values=['Summer','Winter','Both']
# df['Better_Events']=np.select(cond,values)
# print(df)

def get_signals(features):
    signals_df = pd.DataFrame()
    signals_df['volume delta'] = features['Volume'].dropna().pct_change()
    signals_df['bb std delta'] = features['bollinger_std'].dropna().pct_change()
    signals_df['rvol delta'] = features['rvol'].dropna().pct_change()
    signals_df['option rvol delta'] = features['Option rVol'].dropna().pct_change()
    signals_df['daily returns'] = features['daily returns'].dropna()

    return signals_df


### Pass the stock underlying dataframe in this


def get_underlying_volume(uf, symbol):

    bollinger_window=  20
    
    volume_df=uf[uf['Name']==symbol]
    #volume_df = volume_df.merge(volume, left_index=True, right_index = True)

     # more steps feature engineering
    volume_df.loc[:, 'bollinger_mid_band'] = volume_df['Close'].rolling(window=bollinger_window).mean()
    volume_df.loc[:, 'bollinger_std'] = volume_df['Close'].rolling(window=bollinger_window).std()
    
     #Calculate upper and lowers bands of bollinger band
     
    volume_df['bollinger_upper_band']  = volume_df['bollinger_mid_band'] + (volume_df['bollinger_std'] * 1)
    volume_df['bollinger_lower_band']  = volume_df['bollinger_mid_band'] - (volume_df['bollinger_std'] * 1)

     #Calculate bollinger band trading signal
    volume_df['bollinger_long'] = np.where(volume_df['Close'] < volume_df['bollinger_lower_band'], 1.0, 0.0)
    volume_df['bollinger_short'] = np.where(volume_df['Close'] > volume_df['bollinger_upper_band'], -1.0, 0.0)
    volume_df['bollinger_signal'] = volume_df['bollinger_long'] + volume_df['bollinger_short']
    
    #construct relative volume ratio and append to volume
    volume_ratio = 20

     #calculate rvol and 1 deviation of a 20 day rvol
    volume_df['rvol'] = volume_df['Volume']/ volume_df['Volume'].rolling(window = volume_ratio).mean()
    volume_df['rvol SD'] = volume_df['rvol'].rolling(window = volume_ratio).std()/2+ volume_df['rvol'].rolling(window = volume_ratio).mean()
   
    #create rvol signal where rvol > 1 SD rvol
    volume_df['rvol signal'] = np.where(volume_df['rvol'] > volume_df['rvol SD'], 1.0, 0.0)

     #create options rVol signal
    volume_ratio = 20

     #create daily returns column and Actual Value column
    volume_df['daily returns'] = volume_df['Close'].dropna().pct_change()

     #drop nans due to BB/rvol calculations
    volume_df = volume_df.dropna()
    
    trading_signals = volume_df
    trading_signals['volume delta'] = volume_df['Volume'].dropna().pct_change()
    trading_signals['bb std delta'] = volume_df['bollinger_std'].dropna().pct_change()
    trading_signals['rvol delta'] = volume_df['rvol'].dropna().pct_change()
#     #add daily returns as target
    trading_signals['daily returns'] = volume_df['daily returns'].dropna()

    trading_signals= trading_signals.fillna(value = 0)
    trading_signals= trading_signals.replace([np.inf, -np.inf], 0.0)
    
    return trading_signals

###Merge the option and underlying volume dfs, and create extra signals

def merge(udf,odf):
    combined = udf.merge(odf, left_index = True, right_index = True, how = 'inner')
    volume_ratio =  20
    combined['Option rVol SD'] = combined['rvol'].rolling(window = volume_ratio).std()/2+ combined['rvol'].rolling(window = volume_ratio).mean()

    combined['rvol signal'] = np.where(combined['rvol'] > combined['rvol SD'], 1.0, 0.0)

    combined['Option rVol signal'] = np.where(combined['rvol'] > combined['rvol SD'], 1.0, 0.0)
    return combined

#trading_signals = merge(underlying_volume, option_volume)

inm = -.05
atm = -.01
nr1m = .03
nr2m = .07
outm = .11
fotm =.18
x3w = 15
x2w = 10
x1w = 5
x4w = 20
x5w = 25

times = [x1w, x2w, x3w, x4w, x5w]
variables_time = ['x1w', 'x2w', 'x3w', 'x4w', 'x5w']
perimeter = [inm, atm, nr1m, nr2m, outm, fotm]
variables_perimeter = ['inm', 'atm', 'nr1m', 'nr2m', 'outm', 'fotm']
def create_call_targets(df):
    
    df = df.loc[df['PutCall'] == 'call']
    for i in times:
        for j in perimeter:
            df[f'{j}%_{i}days'] = (df['time delta'] < np.timedelta64(i, 'D')) & (df['time delta'] > np.timedelta64((i-5), 'D'))\
            & (df['% out'] > j) & (df['% out'] < (j - .04))
    return df

def create_put_targets(df):
    
    df = df.loc[df['PutCall'] == 'put']
    for i in times:
        for j in perimeter:
            df[f'{j}%_{i}days'] = (df['time delta'] < np.timedelta64(i, 'D')) & (df['time delta'] > np.timedelta64((i-5), 'D'))\
            & (df['% out'] > (j * -1)) & (df['% out'] < ((j * -1) + .04))
    return df


#Join RF and LSTM
def join_signals(df, features_signal, lstm_signal, trial_lstm, shifted_lstm, rf_signal):
    lstm_signal = lstm_signal[['LSTM Predicted Signal']].copy()#add 'DataDate'
    lstm_signal = lstm_signal.rename(columns = {'LSTM Predicted Signal' : 'LSTM'})
    
    rf_signal = rf_signal[['RF Predicted Value']].copy()#add 'DataDate'
    rf_signal = rf_signal.rename(columns={'RF Predicted Value': 'RF'})
    
    trial = trial_lstm[['LSTM Predicted Signal']].copy()
    trial = trial.rename(columns={'LSTM Predicted Signal': 'LSTM.voltrack'})
    

    
    shifted = shifted_lstm[['LSTM Predicted Signal']].copy()
    shifted = shifted.rename(columns={'LSTM Predicted Signal': 'LSTM.shifted'})

    features = features_signal[['Put signals', 'Call signals', 'bollinger_signal','Option rVol signal', 'rvol signal']]
#     signal = signal.set_index(pd.to_datetime(signal.DataDate, infer_datetime_format=True))
    #df = df.set_index(pd.to_datetime(df.DataDate, infer_datetime_format=True))
    df = df.set_index(pd.to_datetime(df.DataDate, infer_datetime_format=True))

    first_result = df.merge(lstm_signal, left_index = True, right_index = True, how= 'inner')
    second_result = first_result.merge(features, left_index = True, right_index = True, how= 'inner')
    third_result = second_result.merge(rf_signal, left_index = True, right_index = True, how= 'inner') 
    fourth_result = third_result.merge(trial, left_index = True, right_index = True, how= 'inner')
    result = fourth_result.merge(shifted, left_index = True, right_index = True, how= 'inner')

   # result = result.drop(columns = ['AskSize', 'BidPrice', 'BidSize', 'LastPrice', 'Volume', 'openinterest', 'Date', 'DataDate'])
    return result

###Create a function to make new directories to save the files
def make_symbol_directory(symbol, sector):
    # Parent Directory path  
    parent_dir = "C:\\Users\\austi\\OneDrive\\Desktop\\Clean Start\master directory\trials"

    # Path  
    path = os.path.join(parent_dir, sector, symbol)  

    # Create the directory  
    os.makedirs(path)  
    print("Directory '% s' created" % symbol)  

def return_list(sector):
    if sector == 'finance':
        return finance_list
    if sector == 'etf':
        return etf_list
    if sector == 'materials':
        return materials_list
    if sector == 'media':
        return media_list
    if sector == 'tech':
        return tech_list
    if sector == 'retail':
        return retail_list
    if sector == 'military':
        return military_list
    if sector == 'food':
        return food_list
               
def tuple_list():
    perimeter = [-.1, -.05, -.01, .04, .06, .1]
    final = []

    shift5 = [5] * len(perimeter)
    shift5 = list(zip(shift5, perimeter))
    final.extend(shift5)

    shift10 = [10] * len(perimeter)
    shift10 = list(zip(shift10, perimeter))
    final.extend(shift10)


    shift15 = [15] * len(perimeter)
    shift15 = list(zip(shift15, perimeter))
    final.extend(shift15)


    shift20 = [20] * len(perimeter)
    shift20 = list(zip(shift20, perimeter))
    final.extend(shift20)

    return final
        

def lstm_call_targets(df, tuplelist):
    
    df = df.loc[df['PutCall'] == 'call']
    for i in tuplelist:
        df[f'{i[0]}days.{i[1]}%'] = (df['time delta'] < np.timedelta64(i[0], 'D')) & (df['time delta'] > np.timedelta64((i[0]-5), 'D')) \
        & (df['% out'] > i[1]) & (df['% out'] < (i[1] + .04))
   
    return df


def lstm_put_targets(df, tuplelist):
    
    df = df.loc[df['PutCall'] == 'put']
    for i in tuplelist:
        df[f'{i[0]}days.{i[1]}%'] = (df['time delta'] < np.timedelta64(i[0], 'D')) & (df['time delta'] > np.timedelta64((i[0]-5), 'D')) \
        & (df['% out'] > (i[1] * -1)) & (df['% out'] < ((i[1] * -1) + .04))
   
    return df

def lstm_perimeters(df, lstm, shift):
    total_max = df['error'].iloc[-3]
    total_75 = df['error'].iloc[-4]

    positive_max = df['positive_error'].iloc[-3]
    positive_75 = df['positive_error'].iloc[-4]
    
    negative_max = df['negative_error'].iloc[-3]
    negative_75 = df['negative_error'].iloc[-4]
    percent_list = [total_max, total_75, positive_max, positive_75, negative_max, negative_75]
    percent_names = ['total_max', 'total_75', 'positive_max', 'positive_75', 'negative_max', 'negative_75']
    shift = [shift] * len(percent_list)
    final_list = list(zip(percent_names, percent_list, shift))
    df = pd.DataFrame(final_list, columns =['Names', 'Names', 'Values'])
    df.to_csv(f'master_directory/lstm_percent_reference/{each_symbol}_{lstm}.csv')
    return final_list      
           
           
def lstm_process(df):   
    df['error'] = df['Actual'] - df['Predicted']
    df.loc[df['Predicted'] > 0, 'positive_predictions']= df.loc[df['Predicted'] > 0, 'Predicted']
    df.loc[df['Predicted'] > 0, 'positive_actual']= df.loc[df['Predicted'] > 0, 'Actual']

    df.loc[df['Predicted'] < 0, 'negative_predictions']= df.loc[df['Predicted'] < 0, 'Predicted']
    df.loc[df['Predicted'] < 0, 'negative_actual']= df.loc[df['Predicted'] < 0, 'Actual']

    df['positive_error'] = df['positive_predictions'] - df['positive_actual']
    df['negative_error'] = df['negative_predictions'] - df['negative_actual']

    df = df.append(df.describe())
    df.tail(10)
    df = df.round(4)
    return df

def count(df):
    ndarray = df.to_numpy()
    ndarray
    positive_row = np.sum(np.array(ndarray) > 0, axis = 0)
    negative_row = np.sum(np.array(ndarray) < 0, axis = 0)
    df.loc[len(df)] = positive_row
    df.loc[len(df)] = negative_row
    df.rename({df.index[-2]: 'positives'}, inplace=True)
    df.rename({df.index[-1]: 'negatives'}, inplace=True)

    return df

###This function will input a list of symbols and save the underlying stock information       
def get_underlying(Symbols):
    start_date = '2016-01-01'
    end_date = '2020-12-31'
# create empty underlying
    underlying = pd.DataFrame()
# iterate odfer each symbol
    for i in Symbols:  


        try:
            # download the stock price 
            stock = []
            stock = yf.download(i,start=start_date, end=end_date, progress=False)

            # append the individual stock prices 
            if len(stock) == 0:
                None
            else:
                stock['Name']=i
                underlying = underlying.append(stock,sort=False)
                underlying= underlying.drop(['Open', 'High', 'Low', 'Adj Close'], axis=1)
        except Exception:
            None
    return underlying
ttt

def append_rows(df):
    data_list = []
    for item in df.transpose():
        data_list.append(df.transpose()[item].to_list())
    original = list(df)
    append_cost = 'cost_'
    append_return = 'return_'
    cost = [append_cost + sub for sub in original]
    returns = [append_return + sub for sub in original]
    complete = original + cost + returns
    complete.pop(len(original))
    complete.pop(len(original)*2-1)
    data_frame = pd.DataFrame(columns = complete)

    for i in range(len(data_list)):
        if 'profit' in data_list[i][0]:
            #print('True')        
            temp_list = data_list[i]    
            for j in range(i+1,i+3,1):
                for val in data_list[j]:
                    if type(val) == str:
                        continue
                    temp_list.append(val)
            data_frame.loc[len(data_frame)] = temp_list
        else:
            if 'cost' in data_list[i][0] or 'return' in data_list[i][0]:
                continue
            for k in range(len(data_frame.columns) - len(data_list[i])):
                data_list[i].append("")
            data_frame.loc[len(data_frame)] = data_list[i]

    return data_frame
    
def downcast_dtypes(df):
    _start = df.memory_usage(deep=True).sum()/1024 ** 2
    float_cols = [c for c in df if df[c].dtype == 'float64']
    int_cols = [c for c in df if df[c].dtype in ['int64', 'int32']]
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols] = df[int_cols].astype(np.int16)
    _end = df.memory_usage(deep=True).sum() / 1024 ** 2
    saved = (_start - _end) / _start * 100
    print(f'saved {saved:.2f}%')
    return df




def delete_columns(df):
    prefix = ['spsp_', 'sp_', 'bpsp_', 'bp_', 'bc_', 'sc_', 'scsp_', 'bcsp_', '']
    column = ['LSTM.voltrack.', 'RF.LSTM.shifted' ]
    for p in prefix:
        for c in column:
            print(f'deleting- {p}{c}')
            unwanted = df.columns[df.columns.str.startswith(f'{p}{c}')]
            df.drop(unwanted, axis = 1, inplace = True)
    return df
                                                     

def organize(aa):
#     symbol = aa['Symbol'].unique()
 #   aa = downcast_dtypes(aa)
    
    print('organizing')
    
    aa = aa.drop(aa.columns[:62], axis=1)
    aa = aa.dropna(how = 'all')
    aa = delete_columns(aa)
   
    aa = aa.append(aa.describe(include=[np.number], exclude = [object]))
    aa = aa.append(aa[:-8].agg(['sum']))
        
    print('counting')
    aa = count(aa)
    
    print('summarizing')
    aa = summarize(aa)
    
    print('appending rows')    
    aa = aa.reset_index()
    aa = append_rows(aa)
    aa = split_index(aa)
    aa = summary_columns(aa)
    return aa    
    
  

def string(s):
    s = str(s)
    remove ='"]['
    q = "'"
    for r in remove:
        s = s.replace(r, "")
    s = s.replace(q, "")  
    return s


def split_index(df):
    #df = df.reset_index()
    new = df["index"].str.split("_", n = 6, expand = True)
    df['strategy'] = new[0]
    df['signal'] = new[1]
    df['perimeter'] = new[2]
    df['symbol'] = new[4]
    df = df.set_index('index')
    return df

def make_d(df, folder, subfolder, file):
    # Parent Directory path  
    df = round(df, 3)
    parent_dir = "C:\\Users\\austi\\OneDrive\\Desktop\\gorgeous george\\updated_master"

    # Path  
    path = os.path.join(parent_dir, folder, subfolder)  
    
    try:
        os.makedirs(path) 
#         print("folder created ")
        df.to_csv(fr"{path}\{file}.csv")
    except FileExistsError:
#         print("folder already exists")
        df.to_csv(fr"{path}\{file}.csv")

def summary_columns(aa):
    aa['ROI'] = aa['sum'] / aa['cost_sum']
    aa['profit_sum'] = aa['sum']
    aa = aa[['ROI', 'profit_sum', 'cost_sum', 'positives', 'negatives', 'min', 'mean', '75%', '50%', '25%', 'cost_mean',\
             'return_mean', 'return_std', 'return_max', 'return_75%', 'return_50%', 'symbol', 'strategy', 'signal', 'perimeter']]
    aa = aa.sort_values('ROI', ascending = False)
    return aa

    
def save(df, sector):
    
    symbol = df['symbol'].unique()
    symbol = string(symbol)

    strategy = df['strategy'].unique()
    strategy = string(strategy)
    make_d(df, 'all_results', strategy, f'{strategy}_{symbol}')
    df.to_csv(f'updated_master\\all_results\\{strategy}_{symbol}.csv')
    make_d(df, strategy, 'all_results', symbol)
    make_d(df, strategy, sector, symbol)
    #df = df.sort_values()
    

    for signal in df['signal'].unique():
        ty = df.loc[(df['signal'] == signal)]
        signal = string(signal)
        make_d(ty, signal, 'all_results', symbol)
        make_d(ty, signal, strategy, symbol)
        make_d(ty, sector, signal, symbol)
        make_d(ty, signal, sector, symbol)
               
        for perimeter in ty['perimeter'].unique():
            
            mk = ty.loc[(ty['signal'] == signal) & (ty['perimeter'] == perimeter)]
            make_d(mk, signal, perimeter, symbol)
            make_d(mk, strategy, signal, perimeter)
            make_d(mk, sector, strategy, perimeter)


def get_home(df, sector):
    #df = split_index(df)
    save(df, sector)
    
    
    
def selling_puts_spread_LSTM_joined(aa, perimeter, asignal, bsignal): 
    
    symbol = aa['Symbol'].unique()
    symbol = string(symbol)
    if asignal != bsignal:

        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1) & (aa["LSTM"] == 1), f'spsp_LSTM.{asignal}.{bsignal}_{perimeter}_profit_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1) & (aa["LSTM"] == 1), 'spread_profit'].astype(float)

        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1) & (aa["LSTM"] == 1), f'spsp_LSTM.{asignal}.{bsignal}_{perimeter}_cost_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1) & (aa["LSTM"] == 1), 'spread_collateral'].astype(float)



        aa[f'spsp_LSTM.{asignal}.{bsignal}_{perimeter}_return_{symbol}'] = aa[f'spsp_LSTM.{asignal}.{bsignal}_{perimeter}_profit_{symbol}']  /  aa[f'spsp_LSTM.{asignal}.{bsignal}_{perimeter}_cost_{symbol}']    

        
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1), f'spsp_{asignal}.{bsignal}_{perimeter}_profit_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1), 'spread_profit'].astype(float)

        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1), f'spsp_{asignal}.{bsignal}_{perimeter}_cost_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1), 'spread_collateral'].astype(float)
        
        aa[f'spsp_{asignal}.{bsignal}_{perimeter}_return_{symbol}'] = aa[f'spsp_{asignal}.{bsignal}_{perimeter}_profit_{symbol}']  /  aa[f'spsp_{asignal}.{bsignal}_{perimeter}_cost_{symbol}']    
    
    
    if asignal == bsignal:
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1) & (aa["LSTM"] == 1), f'spsp_LSTM.{asignal}_{perimeter}_profit_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1) & (aa["LSTM"] == 1), 'spread_profit'].astype(float)

        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1) & (aa["LSTM"] == 1), f'spsp_LSTM.{asignal}_{perimeter}_cost_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1) & (aa["LSTM"] == 1), 'spread_collateral'].astype(float)
        
        aa[f'spsp_LSTM.{asignal}_{perimeter}_return_{symbol}'] = aa[f'spsp_LSTM.{asignal}_{perimeter}_profit_{symbol}'] / aa[f'spsp_LSTM.{asignal}_{perimeter}_cost_{symbol}']
        
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1), f'spsp_{asignal}_{perimeter}_profit_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1), 'spread_profit'].astype(float)

        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1), f'spsp_{asignal}_{perimeter}_cost_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1), 'spread_collateral'].astype(float)
        
        aa[f'spsp_{asignal}_{perimeter}_return_{symbol}'] = aa[f'spsp_{asignal}_{perimeter}_profit_{symbol}']  /  aa[f'spsp_{asignal}_{perimeter}_cost_{symbol}']    

    aa.loc[(aa[perimeter] == True) & (aa["LSTM"] == 1), f'spsp_LSTM_{perimeter}_profit_{symbol}'] = \
    aa.loc[(aa[perimeter] == True) & (aa["LSTM"] == 1), 'spread_profit'].astype(float)

    aa.loc[(aa[perimeter] == True) & (aa["LSTM"] == 1), f'spsp_LSTM_{perimeter}_cost_{symbol}'] = \
    aa.loc[(aa[perimeter] == True) & (aa["LSTM"] == 1), 'spread_collateral'].astype(float)

    aa[f'spsp_LSTM_{perimeter}_return_{symbol}'] = aa[f'spsp_LSTM_{perimeter}_profit_{symbol}'] / aa[f'spsp_LSTM_{perimeter}_cost_{symbol}']
    
    return aa

def selling_puts_LSTM_joined(aa, perimeter, asignal, bsignal): 
    symbol = aa['Symbol'].unique()
    symbol = string(symbol)
    
    if asignal != bsignal:

        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1) & (aa["LSTM"] == 1), f'sp_LSTM.{asignal}.{bsignal}_{perimeter}_profit_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1) & (aa["LSTM"] == 1), 'profit'].astype(float)

        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1) & (aa["LSTM"] == 1), f'sp_LSTM.{asignal}.{bsignal}_{perimeter}_cost_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1) & (aa["LSTM"] == 1), 'AskPrice'].astype(float)

        aa[f'sp_LSTM.{asignal}.{bsignal}_{perimeter}_return_{symbol}'] = aa[f'sp_LSTM.{asignal}.{bsignal}_{perimeter}_profit_{symbol}']  /  aa[f'sp_LSTM.{asignal}.{bsignal}_{perimeter}_cost_{symbol}']    

        
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1), f'sp_{asignal}.{bsignal}_{perimeter}_profit_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1), 'profit'].astype(float)

        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1), f'sp_{asignal}.{bsignal}_{perimeter}_cost_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1), 'AskPrice'].astype(float)
        
        aa[f'sp_{asignal}.{bsignal}_{perimeter}_return_{symbol}'] = aa[f'sp_{asignal}.{bsignal}_{perimeter}_profit_{symbol}']  /  aa[f'sp_{asignal}.{bsignal}_{perimeter}_cost_{symbol}']    
    
    
    if asignal == bsignal:
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1) & (aa["LSTM"] == 1), f'sp_LSTM.{asignal}_{perimeter}_profit_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1) & (aa["LSTM"] == 1), 'profit'].astype(float)

        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1) & (aa["LSTM"] == 1), f'sp_LSTM.{asignal}_{perimeter}_cost_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1) & (aa["LSTM"] == 1), 'AskPrice'].astype(float)
        
        aa[f'sp_LSTM.{asignal}_{perimeter}_return_{symbol}'] = aa[f'sp_LSTM.{asignal}_{perimeter}_profit_{symbol}'] / aa[f'sp_LSTM.{asignal}_{perimeter}_cost_{symbol}']

        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1), f'sp_{asignal}_{perimeter}_profit_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1), 'profit'].astype(float)

        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1), f'sp_{asignal}_{perimeter}_cost_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1), 'AskPrice'].astype(float)
        
        aa[f'sp_{asignal}_{perimeter}_return_{symbol}'] = aa[f'sp_{asignal}_{perimeter}_profit_{symbol}']  /  aa[f'sp_{asignal}_{perimeter}_cost_{symbol}']    

        
    aa.loc[(aa[perimeter] == True) & (aa["LSTM"] == 1), f'sp_LSTM_{perimeter}_profit_{symbol}'] = \
    aa.loc[(aa[perimeter] == True) & (aa["LSTM"] == 1), 'profit'].astype(float)

    aa.loc[(aa[perimeter] == True) & (aa["LSTM"] == 1), f'sp_LSTM_{perimeter}_cost_{symbol}'] = \
    aa.loc[(aa[perimeter] == True) & (aa["LSTM"] == 1), 'AskPrice'].astype(float)

    
    aa[f'sp_LSTM_{perimeter}_return_{symbol}'] = aa[f'sp_LSTM_{perimeter}_profit_{symbol}'] / aa[f'sp_LSTM_{perimeter}_cost_{symbol}']
    
    return aa


def buying_puts_spreads_LSTM_joined(aa, perimeter, asignal, bsignal):
    
    symbol = aa['Symbol'].unique()
    symbol = string(symbol)
    
    aa['spread_profit'] = aa['spread_profit'] * -1
    
    if asignal != bsignal:

        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1) & (aa["LSTM"] == -1), f'bpsp_LSTM.{asignal}.{bsignal}_{perimeter}_profit_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1) & (aa["LSTM"] == -1), 'spread_profit'].astype(float)

        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1) & (aa["LSTM"] == -1), f'bpsp_LSTM.{asignal}.{bsignal}_{perimeter}_cost_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1) & (aa["LSTM"] == -1), 'spread_collateral'].astype(float)

        aa[f'bpsp_LSTM.{asignal}.{bsignal}_{perimeter}_return_{symbol}'] = aa[f'bpsp_LSTM.{asignal}.{bsignal}_{perimeter}_profit_{symbol}']  /  aa[f'bpsp_LSTM.{asignal}.{bsignal}_{perimeter}_cost_{symbol}']    

        
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1), f'bpsp_{asignal}.{bsignal}_{perimeter}_profit_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1), 'spread_profit'].astype(float)

        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1), f'bpsp_{asignal}.{bsignal}_{perimeter}_cost_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1), 'spread_collateral'].astype(float)
        
        aa[f'bpsp_{asignal}.{bsignal}_{perimeter}_return_{symbol}'] = aa[f'bpsp_{asignal}.{bsignal}_{perimeter}_profit_{symbol}']  /  aa[f'bpsp_{asignal}.{bsignal}_{perimeter}_cost_{symbol}']    
    
    
    if (asignal == bsignal):
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1) & (aa["LSTM"] == -1), f'bpsp_LSTM.{asignal}_{perimeter}_profit_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1) & (aa["LSTM"] == -1), 'spread_profit'].astype(float)

        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1) & (aa["LSTM"] == -1), f'bpsp_LSTM.{asignal}_{perimeter}_cost_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1) & (aa["LSTM"] == -1), 'spread_collateral'].astype(float)
        
        aa[f'bpsp_LSTM.{asignal}_{perimeter}_return_{symbol}'] = aa[f'bpsp_LSTM.{asignal}_{perimeter}_profit_{symbol}']  /  aa[f'bpsp_LSTM.{asignal}_{perimeter}_cost_{symbol}']    

        
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1), f'bpsp_{asignal}_{perimeter}_profit_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1), 'spread_profit'].astype(float)

        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1), f'bpsp_{asignal}_{perimeter}_cost_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1), 'spread_collateral'].astype(float)
        
        aa[f'bpsp_{asignal}_{perimeter}_return_{symbol}'] = aa[f'bpsp_{asignal}_{perimeter}_profit_{symbol}']  /  aa[f'bpsp_{asignal}_{perimeter}_cost_{symbol}']    

   

    aa.loc[(aa[perimeter] == True) & (aa['profit'] > 0) & (aa["LSTM"] == -1), f'bpsp_LSTM_{perimeter}_profit_{symbol}'] =\
    aa.loc[(aa[perimeter] == True) & (aa['profit'] > 0) & (aa["LSTM"] == -1), 'spread_profit'].astype(float)

    aa.loc[(aa[perimeter] == True) & (aa['profit'] < 0) & (aa["LSTM"] == -1), f'bpsp_LSTM_{perimeter}_cost_{symbol}'] = \
    aa.loc[(aa[perimeter] == True) & (aa['profit'] < 0) & (aa["LSTM"] == -1), 'spread_collateral'].astype(float)

    aa[f'bpsp_LSTM_{perimeter}_return_{symbol}'] = aa[f'bpsp_LSTM_{perimeter}_profit_{symbol}'] / aa[f'bpsp_LSTM_{perimeter}_cost_{symbol}']
        
    return aa


def buying_puts_LSTM_joined(aa, perimeter, asignal, bsignal):
    
    symbol = aa['Symbol'].unique()
    symbol = string(symbol)
    
    aa['profit'] = aa['profit'] * -1
    
    if asignal != bsignal:

        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1) & (aa["LSTM"] == -1), f'bp_LSTM.{asignal}.{bsignal}_{perimeter}_profit_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1) & (aa["LSTM"] == -1), 'profit'].astype(float)

        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1) & (aa["LSTM"] == -1), f'bp_LSTM.{asignal}.{bsignal}_{perimeter}_cost_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1) & (aa["LSTM"] == -1), 'AskPrice'].astype(float)


        aa[f'bp_LSTM.{asignal}.{bsignal}_{perimeter}_return_{symbol}'] = aa[f'bp_LSTM.{asignal}.{bsignal}_{perimeter}_profit_{symbol}']  /  aa[f'bp_LSTM.{asignal}.{bsignal}_{perimeter}_cost_{symbol}']    

        
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1), f'bp_{asignal}.{bsignal}_{perimeter}_profit_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1), 'profit'].astype(float)

        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1), f'bp_{asignal}.{bsignal}_{perimeter}_cost_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1), 'AskPrice'].astype(float)
        
        aa[f'bp_{asignal}.{bsignal}_{perimeter}_return_{symbol}'] = aa[f'bp_{asignal}.{bsignal}_{perimeter}_profit_{symbol}']  /  aa[f'bp_{asignal}.{bsignal}_{perimeter}_cost_{symbol}']    
    
    
    if (asignal == bsignal):
        
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1) & (aa["LSTM"] == -1), f'bp_LSTM.{asignal}_{perimeter}_profit_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1) & (aa["LSTM"] == -1), 'profit'].astype(float)

        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1) & (aa["LSTM"] == -1), f'bp_LSTM.{asignal}_{perimeter}_cost_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1) & (aa["LSTM"] == -1), 'AskPrice'].astype(float)
        
        aa[f'bp_LSTM.{asignal}_{perimeter}_return_{symbol}'] = aa[f'bp_LSTM.{asignal}_{perimeter}_profit_{symbol}']  /  aa[f'bp_LSTM.{asignal}_{perimeter}_cost_{symbol}']    


        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1), f'bp_{asignal}_{perimeter}_profit_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1), 'profit'].astype(float)

        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1), f'bp_{asignal}_{perimeter}_cost_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1), 'AskPrice'].astype(float)
        
        aa[f'bp_{asignal}_{perimeter}_return_{symbol}'] = aa[f'bp_{asignal}_{perimeter}_profit_{symbol}']  /  aa[f'bp_{asignal}_{perimeter}_cost_{symbol}']    

   


    aa.loc[(aa[perimeter] == True) & (aa['profit'] > 0) & (aa["LSTM"] == -1), f'bp_LSTM_{perimeter}_profit_{symbol}'] =\
    aa.loc[(aa[perimeter] == True) & (aa['profit'] > 0) & (aa["LSTM"] == -1), 'profit'].astype(float)

    aa.loc[(aa[perimeter] == True) & (aa['profit'] < 0) & (aa["LSTM"] == -1), f'bp_LSTM_{perimeter}_cost_{symbol}'] = \
    aa.loc[(aa[perimeter] == True) & (aa['profit'] < 0) & (aa["LSTM"] == -1), 'AskPrice'].astype(float)

    aa[f'bp_LSTM_{perimeter}_return_{symbol}'] = aa[f'bp_LSTM_{perimeter}_profit_{symbol}'] / aa[f'bp_LSTM_{perimeter}_cost_{symbol}']
        
    return aa

def bull_puts_results(df):
    perimeters = ['5days.-0.1%',
    '5days.-0.05%','5days.-0.01%','5days.0.04%','5days.0.06%','5days.0.1%','10days.-0.1%','10days.-0.05%',\
                  '10days.-0.01%','10days.0.04%','10days.0.06%','10days.0.1%','15days.-0.1%','15days.-0.05%',\
                  '15days.-0.01%','15days.0.04%','15days.0.06%','15days.0.1%','20days.-0.1%','20days.-0.05%',\
                  '20days.-0.01%','20days.0.04%','20days.0.06%','20days.0.1%']

    signals = ['LSTM.voltrack', 'LSTM.shifted', 'RF']


    for i in perimeters:
        for j in signals:
            for k in signals:
                results = selling_puts_LSTM_joined(df, i, j, k)
    return results

def bull_puts_spread_results(df):
    perimeters = ['5days.-0.1%',
    '5days.-0.05%','5days.-0.01%','5days.0.04%','5days.0.06%','5days.0.1%','10days.-0.1%','10days.-0.05%',\
                  '10days.-0.01%','10days.0.04%','10days.0.06%','10days.0.1%','15days.-0.1%','15days.-0.05%',\
                  '15days.-0.01%','15days.0.04%','15days.0.06%','15days.0.1%','20days.-0.1%','20days.-0.05%',\
                  '20days.-0.01%','20days.0.04%','20days.0.06%','20days.0.1%']

    signals = ['LSTM.voltrack', 'LSTM.shifted', 'RF']


    for i in perimeters:
        for j in signals:
            for k in signals:
                results = selling_puts_spread_LSTM_joined(df, i, j, k)
    return results

def bear_puts_results(df):
    perimeters = ['5days.-0.1%',
    '5days.-0.05%','5days.-0.01%','5days.0.04%','5days.0.06%','5days.0.1%','10days.-0.1%','10days.-0.05%',\
                  '10days.-0.01%','10days.0.04%','10days.0.06%','10days.0.1%','15days.-0.1%','15days.-0.05%',\
                  '15days.-0.01%','15days.0.04%','15days.0.06%','15days.0.1%','20days.-0.1%','20days.-0.05%',\
                  '20days.-0.01%','20days.0.04%','20days.0.06%','20days.0.1%']

    signals = [ 'LSTM.voltrack', 'LSTM.shifted']


    for i in perimeters:
        for j in signals:
            for k in signals:
                results = buying_puts_LSTM_joined(df, i, j, k)
    return results

def bear_puts_spreads_results(df):
    perimeters = ['5days.-0.1%',
    '5days.-0.05%','5days.-0.01%','5days.0.04%','5days.0.06%','5days.0.1%','10days.-0.1%','10days.-0.05%',\
                  '10days.-0.01%','10days.0.04%','10days.0.06%','10days.0.1%','15days.-0.1%','15days.-0.05%',\
                  '15days.-0.01%','15days.0.04%','15days.0.06%','15days.0.1%','20days.-0.1%','20days.-0.05%',\
                  '20days.-0.01%','20days.0.04%','20days.0.06%','20days.0.1%']

    signals = ['LSTM.voltrack', 'LSTM.shifted']


    for i in perimeters:
        for j in signals:
            for k in signals:
                results = buying_puts_spreads_LSTM_joined(df, i, j, k)
    return results

def bear_puts_finisher(df, sector):
    tp(start)
    print('getting bear puts results')
    results = bear_puts_results(df)
    
    print('getting bear puts spreads')
    spread_results = bear_puts_spreads_results(df)
    
    print('organizing bear puts')
    organized = organize(results)
    
    print('organizing bear puts spreads')
    organized_spreads = organize(spread_results)
    
    tp(start)
    print('sending bear puts home')
    get_home(organized, sector)
    get_home(organized_spreads, sector)

def bull_puts_finisher(df, sector):
    tp(start)
    print('getting bull puts')
    results = bull_puts_results(df)
    
    print('organizing bull puts')
    organized = organize(results)
    
    print('getting bull puts spreads')
    spread_results = bull_puts_spread_results(df)
    
    print('organizing bull puts spreads')
    organized_spreads = organize(spread_results)
    
    tp(start)
    print('sending bull puts home')
    get_home(organized, sector)
    get_home(organized_spreads, sector)
    
def puts_finisher(df, sector):
    bear_puts_finisher(df, sector)
    bull_puts_finisher(df, sector)

def bull_call_results(df):
    perimeters = ['5days.-0.1%',
    '5days.-0.05%','5days.-0.01%','5days.0.04%','5days.0.06%','5days.0.1%','10days.-0.1%','10days.-0.05%',\
                  '10days.-0.01%','10days.0.04%','10days.0.06%','10days.0.1%','15days.-0.1%','15days.-0.05%',\
                  '15days.-0.01%','15days.0.04%','15days.0.06%','15days.0.1%','20days.-0.1%','20days.-0.05%',\
                  '20days.-0.01%','20days.0.04%','20days.0.06%','20days.0.1%']

    signals = ['LSTM.voltrack', 'LSTM.shifted', 'RF']


    for i in perimeters:
        for j in signals:
            for k in signals:
                results = buying_calls_LSTM_joined(df, i, j, k)
    return results

def bull_call_spread_results(df):
    perimeters = ['5days.-0.1%',
    '5days.-0.05%','5days.-0.01%','5days.0.04%','5days.0.06%','5days.0.1%','10days.-0.1%','10days.-0.05%',\
                  '10days.-0.01%','10days.0.04%','10days.0.06%','10days.0.1%','15days.-0.1%','15days.-0.05%',\
                  '15days.-0.01%','15days.0.04%','15days.0.06%','15days.0.1%','20days.-0.1%','20days.-0.05%',\
                  '20days.-0.01%','20days.0.04%','20days.0.06%','20days.0.1%']

    signals = ['LSTM.voltrack', 'LSTM.shifted', 'RF']


    for i in perimeters:
        for j in signals:
            for k in signals:
                results = buying_calls_spread_LSTM_joined(df, i, j, k)
    return results

def bear_call_results(df):
    perimeters = ['5days.-0.1%',
    '5days.-0.05%','5days.-0.01%','5days.0.04%','5days.0.06%','5days.0.1%','10days.-0.1%','10days.-0.05%',\
                  '10days.-0.01%','10days.0.04%','10days.0.06%','10days.0.1%','15days.-0.1%','15days.-0.05%',\
                  '15days.-0.01%','15days.0.04%','15days.0.06%','15days.0.1%','20days.-0.1%','20days.-0.05%',\
                  '20days.-0.01%','20days.0.04%','20days.0.06%','20days.0.1%']

    signals = ['LSTM.voltrack', 'LSTM.shifted']


    for i in perimeters:
        for j in signals:
            for k in signals:
                results = selling_calls_LSTM_joined(df, i, j, k)
    return results

def bear_call_spreads_results(df):
    perimeters = ['5days.-0.1%',
    '5days.-0.05%','5days.-0.01%','5days.0.04%','5days.0.06%','5days.0.1%','10days.-0.1%','10days.-0.05%',\
                  '10days.-0.01%','10days.0.04%','10days.0.06%','10days.0.1%','15days.-0.1%','15days.-0.05%',\
                  '15days.-0.01%','15days.0.04%','15days.0.06%','15days.0.1%','20days.-0.1%','20days.-0.05%',\
                  '20days.-0.01%','20days.0.04%','20days.0.06%','20days.0.1%']

    signals = ['LSTM.voltrack', 'LSTM.shifted']


    for i in perimeters:
        for j in signals:
            for k in signals:
                results = selling_calls_spreads_LSTM_joined(df, i, j, k)
    return results

def bear_call_finisher(df, sector):
    tp(start)
    print('getting bear call results')
    results = bear_call_results(df)
    
    print('getting bear call spreads results')
    spread_results = bear_call_spreads_results(df)
    
    print('organizing bear call results')
    organized = organize(results)
    
    print('organizing bear call spreads results')
    organized_spreads = organize(spread_results)
    
    tp(start)
    print('sending bear calls home')
    get_home(organized, sector)
    get_home(organized_spreads, sector)

def bull_call_finisher(df, sector):
    tp(start)
    print('getting bull call results')
    results = bull_call_results(df)
    
    print('organizing bull call results')
    organized = organize(results)
    
    print('getting bull call spread results')
    spread_results = bull_call_spread_results(df)
    
    print('organizing bull call spread results')
    organized_spreads = organize(spread_results)
    
    print('sending calls home')
    get_home(organized, sector)
    
    tp(start)
    print('sending calls spreads home')
    get_home(organized_spreads, sector)
    
def call_finisher(df, sector):
    bear_call_finisher(df, sector)
    bull_call_finisher(df, sector)

def count(df):
    ndarray = df.to_numpy()
    ndarray
    positive_row = np.sum(np.array(ndarray) >= 0, axis = 0)
    negative_row = np.sum(np.array(ndarray) < 0, axis = 0)
    positive_row = positive_row - 9
    df.loc[len(df)] = positive_row
    df.loc[len(df)] = negative_row
    df.rename({df.index[-2]: 'positives'}, inplace=True)
    df.rename({df.index[-1]: 'negatives'}, inplace=True)

    return df

def summarize(df):
    
    tail = df[-11:]
    trans = tail.T
    #trans = trans.sort_values(['sum'], ascending = (False))
    trans = trans.round(decimals = 2)
    #trans.to_csv('test_results/new_trans.csv')
    return trans



def buying_calls_LSTM_joined(aa, perimeter, asignal, bsignal): 
    symbol = aa['Symbol'].unique()
    symbol = string(symbol)
    
    
    aa[f'profit'] = aa[f'profit'] * -1
    
    if asignal != bsignal:

        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1) & (aa["LSTM"] == 1), f'bc_LSTM.{asignal}.{bsignal}_{perimeter}_profit_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1) & (aa["LSTM"] == 1), 'profit'].astype(float)

        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1) & (aa["LSTM"] == 1), f'bc_LSTM.{asignal}.{bsignal}_{perimeter}_cost_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1) & (aa["LSTM"] == 1), 'AskPrice'].astype(float)


        aa[f'bc_LSTM.{asignal}.{bsignal}_{perimeter}_return_{symbol}'] = aa[f'bc_LSTM.{asignal}.{bsignal}_{perimeter}_profit_{symbol}']  /  aa[f'bc_LSTM.{asignal}.{bsignal}_{perimeter}_cost_{symbol}']    

        
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1), f'bc_{asignal}.{bsignal}_{perimeter}_profit_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1), 'profit'].astype(float)

        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1), f'bc_{asignal}.{bsignal}_{perimeter}_cost_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1), 'AskPrice'].astype(float)
        
        aa[f'bc_{asignal}.{bsignal}_{perimeter}_return_{symbol}'] = aa[f'bc_{asignal}.{bsignal}_{perimeter}_profit_{symbol}']  /  aa[f'bc_{asignal}.{bsignal}_{perimeter}_cost_{symbol}']    
    
    
    if asignal == bsignal:
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1) & (aa["LSTM"] == 1), f'bc_LSTM.{asignal}_{perimeter}_profit_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1) & (aa["LSTM"] == 1), 'profit'].astype(float)

        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1) & (aa["LSTM"] == 1), f'bc_LSTM.{asignal}_{perimeter}_cost_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1) & (aa["LSTM"] == 1), 'AskPrice'].astype(float)
        
        aa[f'bc_LSTM.{asignal}_{perimeter}_return_{symbol}'] = aa[f'bc_LSTM.{asignal}_{perimeter}_profit_{symbol}'] / aa[f'bc_LSTM.{asignal}_{perimeter}_cost_{symbol}']

        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1), f'bc_{asignal}_{perimeter}_profit_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1), 'profit'].astype(float)

        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1), f'bc_{asignal}_{perimeter}_cost_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1), 'AskPrice'].astype(float)
        
        aa[f'bc_{asignal}_{perimeter}_return_{symbol}'] = aa[f'bc_{asignal}_{perimeter}_profit_{symbol}']  /  aa[f'bc_{asignal}_{perimeter}_cost_{symbol}']    

    aa.loc[(aa[perimeter] == True) & (aa["LSTM"] == 1), f'bc_LSTM_{perimeter}_profit_{symbol}'] = \
    aa.loc[(aa[perimeter] == True) & (aa["LSTM"] == 1), 'profit'].astype(float)

    aa.loc[(aa[perimeter] == True) & (aa["LSTM"] == 1), f'bc_LSTM_{perimeter}_cost_{symbol}'] = \
    aa.loc[(aa[perimeter] == True) & (aa["LSTM"] == 1), 'AskPrice'].astype(float)

    aa[f'bc_LSTM_{perimeter}_return_{symbol}'] = aa[f'bc_LSTM_{perimeter}_profit_{symbol}'] / aa[f'bc_LSTM_{perimeter}_cost_{symbol}']
    
    return aa



def selling_calls_LSTM_joined(aa, perimeter, asignal, bsignal):
    symbol = aa['Symbol'].unique()
    symbol = string(symbol)
    
    if asignal != bsignal:

        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1) & (aa["LSTM"] == -1), f'sc_LSTM.{asignal}.{bsignal}_{perimeter}_profit_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1) & (aa["LSTM"] == -1), 'profit'].astype(float)

        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1) & (aa["LSTM"] == -1), f'sc_LSTM.{asignal}.{bsignal}_{perimeter}_cost_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1) & (aa["LSTM"] == -1), 'AskPrice'].astype(float)


        aa[f'sc_LSTM.{asignal}.{bsignal}_{perimeter}_return_{symbol}'] = aa[f'sc_LSTM.{asignal}.{bsignal}_{perimeter}_profit_{symbol}']  /  aa[f'sc_LSTM.{asignal}.{bsignal}_{perimeter}_cost_{symbol}']    

        
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1), f'sc_{asignal}.{bsignal}_{perimeter}_profit_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1), 'profit'].astype(float)

        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1), f'sc_{asignal}.{bsignal}_{perimeter}_cost_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1), 'AskPrice'].astype(float)
        
        aa[f'sc_{asignal}.{bsignal}_{perimeter}_return_{symbol}'] = aa[f'sc_{asignal}.{bsignal}_{perimeter}_profit_{symbol}']  /  aa[f'sc_{asignal}.{bsignal}_{perimeter}_cost_{symbol}']    
    
    
    if (asignal == bsignal):
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1) & (aa["LSTM"] == -1), f'sc_LSTM.{asignal}_{perimeter}_profit_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1) & (aa["LSTM"] == -1), 'profit'].astype(float)

        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1) & (aa["LSTM"] == -1), f'sc_LSTM.{asignal}_{perimeter}_cost_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1) & (aa["LSTM"] == -1), 'AskPrice'].astype(float)
        
        aa[f'sc_LSTM.{asignal}_{perimeter}_return_{symbol}'] = aa[f'sc_LSTM.{asignal}_{perimeter}_profit_{symbol}']  /  aa[f'sc_LSTM.{asignal}_{perimeter}_cost_{symbol}']    


        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1), f'sc_{asignal}_{perimeter}_profit_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1), 'profit'].astype(float)

        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1), f'sc_{asignal}_{perimeter}_cost_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1), 'AskPrice'].astype(float)
        
        aa[f'sc_{asignal}_{perimeter}_return_{symbol}'] = aa[f'sc_{asignal}_{perimeter}_profit_{symbol}']  /  aa[f'sc_{asignal}_{perimeter}_cost_{symbol}']    

   


    aa.loc[(aa[perimeter] == True) & (aa['profit'] > 0) & (aa["LSTM"] == -1), f'sc_LSTM_{perimeter}_profit_{symbol}'] =\
    aa.loc[(aa[perimeter] == True) & (aa['profit'] > 0) & (aa["LSTM"] == -1), 'profit'].astype(float)

    aa.loc[(aa[perimeter] == True) & (aa['profit'] < 0) & (aa["LSTM"] == -1), f'sc_LSTM_{perimeter}_cost_{symbol}'] = \
    aa.loc[(aa[perimeter] == True) & (aa['profit'] < 0) & (aa["LSTM"] == -1), 'AskPrice'].astype(float)
    
    aa[f'sc_LSTM_{perimeter}_return_{symbol}'] = aa[f'sc_LSTM_{perimeter}_profit_{symbol}'] / aa[f'sc_LSTM_{perimeter}_cost_{symbol}']

    
    return aa



def selling_calls_spreads_LSTM_joined(aa, perimeter, asignal, bsignal): 
    
    symbol = aa['Symbol'].unique()
    symbol = string(symbol)
    
    
    if asignal != bsignal:

        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1) & (aa["LSTM"] == -1), f'scsp_LSTM.{asignal}.{bsignal}_{perimeter}_profit_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1) & (aa["LSTM"] == -1), 'spread_profit'].astype(float)

        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1) & (aa["LSTM"] == -1), f'scsp_LSTM.{asignal}.{bsignal}_{perimeter}_cost_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1) & (aa["LSTM"] == -1), 'spread_collateral'].astype(float)

        aa[f'scsp_LSTM.{asignal}.{bsignal}_{perimeter}_return_{symbol}'] = aa[f'scsp_LSTM.{asignal}.{bsignal}_{perimeter}_profit_{symbol}']  /  aa[f'scsp_LSTM.{asignal}.{bsignal}_{perimeter}_cost_{symbol}']    

        
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1), f'scsp_{asignal}.{bsignal}_{perimeter}_profit_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1), 'spread_profit'].astype(float)

        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1), f'scsp_{asignal}.{bsignal}_{perimeter}_cost_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1), 'spread_collateral'].astype(float)
        
        aa[f'scsp_{asignal}.{bsignal}_{perimeter}_return_{symbol}'] = aa[f'scsp_{asignal}.{bsignal}_{perimeter}_profit_{symbol}']  /  aa[f'scsp_{asignal}.{bsignal}_{perimeter}_cost_{symbol}']    
    
    
    if asignal == bsignal:
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1) & (aa["LSTM"] == -1), f'scsp_LSTM.{asignal}_{perimeter}_profit_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1) & (aa["LSTM"] == -1), 'spread_profit'].astype(float)

        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1) & (aa["LSTM"] == -1), f'scsp_LSTM.{asignal}_{perimeter}_cost_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1) & (aa[bsignal] == -1) & (aa["LSTM"] == -1), 'spread_collateral'].astype(float)
        
        aa[f'scsp_LSTM.{asignal}_{perimeter}_return_{symbol}'] = aa[f'scsp_LSTM.{asignal}_{perimeter}_profit_{symbol}'] / aa[f'scsp_LSTM.{asignal}_{perimeter}_cost_{symbol}']

        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1), f'scsp_{asignal}_{perimeter}_profit_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1), 'spread_profit'].astype(float)

        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1), f'scsp_{asignal}_{perimeter}_cost_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == -1), 'spread_collateral'].astype(float)
        
        aa[f'scsp_{asignal}_{perimeter}_return_{symbol}'] = aa[f'scsp_{asignal}_{perimeter}_profit_{symbol}']  /  aa[f'scsp_{asignal}_{perimeter}_cost_{symbol}']    

    aa.loc[(aa[perimeter] == True) & (aa["LSTM"] == -1), f'scsp_LSTM_{perimeter}_profit_{symbol}'] = \
    aa.loc[(aa[perimeter] == True) & (aa["LSTM"] == -1), 'spread_profit'].astype(float)

    aa.loc[(aa[perimeter] == True) & (aa["LSTM"] == -1), f'scsp_LSTM_{perimeter}_cost_{symbol}'] = \
    aa.loc[(aa[perimeter] == True) & (aa["LSTM"] == -1), 'spread_collateral'].astype(float)

    aa[f'scsp_LSTM_{perimeter}_return_{symbol}'] = aa[f'scsp_LSTM_{perimeter}_profit_{symbol}'] / aa[f'scsp_LSTM_{perimeter}_cost_{symbol}']
    
    
    return aa



def buying_calls_spread_LSTM_joined(aa, perimeter, asignal, bsignal): 
    
    symbol = aa['Symbol'].unique()
    symbol = string(symbol)
    print(f'buying calls spreads for {symbol}')     
    
    
    aa['spread_profit'] = aa['spread_profit'] * -1
    if asignal != bsignal:

        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1) & (aa["LSTM"] == 1), f'bcsp_LSTM.{asignal}.{bsignal}_{perimeter}_profit_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1) & (aa["LSTM"] == 1), 'spread_profit'].astype(float)

        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1) & (aa["LSTM"] == 1), f'bcsp_LSTM.{asignal}.{bsignal}_{perimeter}_cost_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1) & (aa["LSTM"] == 1), 'spread_collateral'].astype(float)


        aa[f'bcsp_LSTM.{asignal}.{bsignal}_{perimeter}_return_{symbol}'] = aa[f'bcsp_LSTM.{asignal}.{bsignal}_{perimeter}_profit_{symbol}']  /  aa[f'bcsp_LSTM.{asignal}.{bsignal}_{perimeter}_cost_{symbol}']    

        
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1), f'bcsp_{asignal}.{bsignal}_{perimeter}_profit_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1), 'spread_profit'].astype(float)

        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1), f'bcsp_{asignal}.{bsignal}_{perimeter}_cost_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1), 'spread_collateral'].astype(float)
        
        aa[f'bcsp_{asignal}.{bsignal}_{perimeter}_return_{symbol}'] = aa[f'bcsp_{asignal}.{bsignal}_{perimeter}_profit_{symbol}']  /  aa[f'bcsp_{asignal}.{bsignal}_{perimeter}_cost_{symbol}']    
    
    
    if asignal == bsignal:
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1) & (aa["LSTM"] == 1), f'bcsp_LSTM.{asignal}_{perimeter}_profit_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1) & (aa["LSTM"] == 1), 'spread_profit'].astype(float)

        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1) & (aa["LSTM"] == 1), f'bcsp_LSTM.{asignal}_{perimeter}_cost_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1) & (aa[bsignal] == 1) & (aa["LSTM"] == 1), 'spread_collateral'].astype(float)

        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1), f'bcsp_{asignal}_{perimeter}_profit_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1), 'spread_profit'].astype(float)

        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1), f'bcsp_{asignal}_{perimeter}_cost_{symbol}'] = \
        aa.loc[(aa[perimeter] == True) & (aa[asignal] == 1), 'spread_collateral'].astype(float)
        
        aa[f'bcsp_{asignal}_{perimeter}_return_{symbol}'] = aa[f'bcsp_{asignal}_{perimeter}_profit_{symbol}']  /  aa[f'bcsp_{asignal}_{perimeter}_cost_{symbol}']    

    aa.loc[(aa[perimeter] == True) & (aa["LSTM"] == 1), f'bcsp_LSTM_{perimeter}_profit_{symbol}'] = \
    aa.loc[(aa[perimeter] == True) & (aa["LSTM"] == 1), 'spread_profit'].astype(float)

    aa.loc[(aa[perimeter] == True) & (aa["LSTM"] == 1), f'bcsp_LSTM_{perimeter}_cost_{symbol}'] = \
    aa.loc[(aa[perimeter] == True) & (aa["LSTM"] == 1), 'spread_collateral'].astype(float)

    aa[f'bcsp_LSTM_{perimeter}_return_{symbol}'] = aa[f'bcsp_LSTM_{perimeter}_profit_{symbol}'] / aa[f'bcsp_LSTM_{perimeter}_cost_{symbol}']
    
    
    return aa


    en = tp(en)




#Pass sector name as a string for option_df, and sector list to underlying_df
def pipeline_features(sector):
    start = time.perf_counter()
    sector_list = return_list(sector)
    underlying_df = get_underlying(sector_list)
    option_df = pd.read_pickle(f'{sector}.pickle')
    en = tp(start)          
    for each_symbol in underlying_df['Name'].unique():
        mf={}
        odf = feature_engineering(option_df, each_symbol)
        opt = get_option_volume(odf, each_symbol)
        undv = get_underlying_volume(underlying_df, each_symbol)
        combined = merge(opt, undv)
        #combined.to_csv(f'{sector}{each_symbol}_features.csv')
        
        en = tp(en)
        
        #Add the lstm function
        lstm_results = robust_lstm.main(combined, 7)
        
        
        en = tp(en)
        
        lstm_shift = robust_lstm.main(combined, 14)
        
        
        en = tp(en)
        
        lstm_trial = working_lstm.main(combined, 7)
        
        
        en = tp(en)
        

        lstm_results = count(lstm_process(lstm_results))
        #lstm_results.to_pickle(f'{sector}{each_symbol}_lstm.pickle')
        #lstm_tuples = lstm_perimeters(lstm_results, 7)
        
        en = tp(en)
        
        
        #Add the rf function
        rf_results = main_rf.main_rf(combined)
        #rf_results.to_pickle(f'{sector}{each_symbol}_rf.pickle')
        
        en = tp(en)
        
        #Join the lstm signals to df
        #the lstm file may need to be altered so it doesn't drop any columns,
        #the format here will not be the same as the csv read before
        #make_symbol_directory(each_symbol, sector)
        signal_ready = join_signals(odf, combined, lstm_results, lstm_trial, lstm_shift, rf_results)
        signal_ready = map_exp(signal_ready)
        
        en = tp(en)
                
        
        #signal_ready.to_csv(f'features_directory\features_{each_symbol}trial_run.csv')
        #calls = create_call_targets(signal_ready)
        #puts = create_put_targets(signal_ready)
        tuples = tuple_list()
        calls = lstm_call_targets(signal_ready, tuples)
        puts = lstm_put_targets(signal_ready, tuples)
        puts = create_spreads(puts, 1)
        calls = create_spreads(calls, -1)
        puts = create_nulls(puts)
        calls = create_nulls(calls)
        
        en = tp(en)
        
#         calls.to_pickle(f'trials/{each_symbol}42calls.pickle')
#         puts.to_pickle(f'trials/{each_symbol}42puts.pickle')
        call_finisher(calls, sector)
        
        en = tp(en)
            
        puts_finisher(puts, sector)
        psutil.virtual_memory()
        
        en = tp(en)
        
#         #signal_ready.to_pickle(f'{sector}_{each_symbol}_try.pickle')
#         mf[each_symbol]= signal_ready
#         mf = pd.concat(mf.values())
#         mf.to_csv(f'{sector}_signals_combined.csv')
#         #save each of the features dataframes to csv
#         #combined.to_csv(f'etf_{each_symbol}_initial.csv')
    return signal_ready
#return mf



In [3]:
etf_list = ['MTUM', 'SPY', 'NDAQ', 'VOO' ]
materials_list = ['VALE', 'RIO', 'BHP']
media_list = ['TWTR', 'NFLX', 'DIS']
tech_list = ['GOOG', 'AAPL', 'FB'] #missing msft and tsla
retail_list = ['BABA', 'AMZN', 'WMT' , 'BBY']
finance_list = ['GS', 'JPM']
military_list = ['BA', 'LMT', 'RTX']
food_list = ['PG', 'SBUX', 'CPG', ]

sectors = ['etf', 'materials', 'media', 'tech', 'retail', 'finance', 'military', 'food']
updated_sectors = ['etf', 'materials', 'tech', 'military', 'food']

from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

start = time.perf_counter()

# for sector in updated_sectors:
#     pipeline_features(sector)
# start = time.process_time()
# your code here    
sample = pipeline_features('food')
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)




Current Time = 08:07:21
current time is 155.5151182
Memory available in MBs: 15826
time since start: ('seconds passed since start:', 0)
current time is 173.3393801
Memory available in MBs: 15676
time since start: ('seconds passed since start:', 3)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 7, 7)              392       
_________________________________________________________________
dropout (Dropout)            (None, 7, 7)              0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 7, 7)              420       
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7)              0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 7)                 420       
______

In [7]:
def test_underlying(Symbols):
    start_date = '2016-01-01'
    end_date = '2020-12-31'
# create empty underlying
    underlying = pd.DataFrame()
# iterate odfer each symbol
  

    stock = []
    stock = yf.download(Symbols,start=start_date, end=end_date, progress=False)

 
    stock['Name']= Symbols
    underlying = underlying.append(stock,sort=False)
    underlying= underlying.drop(['Open', 'High', 'Low', 'Adj Close'], axis=1)
  
    return underlying



def test_pipeline(symbol, df):
    
    sector = 'finance'
    start = time.process_time()
    
    #sector_list = return_list(sector)
    underlying_df = test_underlying(symbol)
    #option_df = pd.read_pickle(f'finance.pickle')
    option_df = downcast_dtypes(df)       
    for each_symbol in underlying_df['Name'].unique():
        mf={}
        odf = feature_engineering(option_df, each_symbol)
        opt = get_option_volume(odf, each_symbol)
        undv = get_underlying_volume(underlying_df, each_symbol)
        combined = merge(opt, undv)
        #combined.to_csv(f'{sector}{each_symbol}_features.csv')
        
           en = tp(en)
        
        #Add the lstm function
        lstm_results = trial_lstm.main(combined, 7)
        
        
           en = tp(en)
        
        lstm_shift = trial_lstm.main(combined, 14)
        
        
           en = tp(en)
        
        lstm_trial = working_lstm.main(combined, 7)
        
        
           en = tp(en)
        

        lstm_results = count(lstm_process(lstm_results))
        #lstm_results.to_pickle(f'{sector}{each_symbol}_lstm.pickle')
        #lstm_tuples = lstm_perimeters(lstm_results, 7)
        
           en = tp(en)
        
        
        #Add the rf function
        rf_results = main_rf.main_rf(combined)
        #rf_results.to_pickle(f'{sector}{each_symbol}_rf.pickle')
        
           en = tp(en)
        
        #Join the lstm signals to df
        #the lstm file may need to be altered so it doesn't drop any columns,
        #the format here will not be the same as the csv read before
        #make_symbol_directory(each_symbol, sector)
        signal_ready = join_signals(odf, combined, lstm_results, lstm_trial, lstm_shift, rf_results)
        signal_ready = map_exp(signal_ready)
        
           en = tp(en)
                
        
        #signal_ready.to_csv(f'features_directory\features_{each_symbol}trial_run.csv')
        #calls = create_call_targets(signal_ready)
        #puts = create_put_targets(signal_ready)
        tuples = tuple_list()
        calls = lstm_call_targets(signal_ready, tuples)
        puts = lstm_put_targets(signal_ready, tuples)
        puts = create_spreads(puts, 1)
        calls = create_spreads(calls, -1)
        puts = create_nulls(puts)
        calls = create_nulls(calls)
        
           en = tp(en)
        
#         calls.to_pickle(f'trials/{each_symbol}42calls.pickle')
#         puts.to_pickle(f'trials/{each_symbol}42puts.pickle')
        call_finisher(calls, sector)
        
           en = tp(en)
            
        puts_finisher(puts, sector)
        psutil.virtual_memory()
        
           en = tp(en)
        
#         #signal_ready.to_pickle(f'{sector}_{each_symbol}_try.pickle')
#         mf[each_symbol]= signal_ready
#         mf = pd.concat(mf.values())
#         mf.to_csv(f'{sector}_signals_combined.csv')
#         #save each of the features dataframes to csv
#         #combined.to_csv(f'etf_{each_symbol}_initial.csv')
    return signal_ready

IndentationError: unexpected indent (<ipython-input-7-05dc4eb8b351>, line 38)

In [ ]:
for col in sample.columns:
    print(col)

In [4]:
etf_list = ['MTUM', 'SPY', 'NDAQ', 'VOO' ]
materials_list = ['VALE', 'RIO', 'BHP']
media_list = ['TWTR', 'NFLX', 'DIS']
tech_list = ['MSFT', 'TSLA', 'GOOG', 'AAPL', 'FB']
retail_list = ['BABA', 'AMZN', 'WMT' , 'BBY']
finance_list = ['GS', 'JPM']
military_list = ['BA', 'LMT', 'RTX']
food_list = ['PG', 'SBUX', 'CPG', ]

sectors = ['etf', 'materials', 'tech', 'retail', 'finance', 'military', 'food'] #missing media

In [6]:
pipeline_features('tech')

current time is 34681.5742697
Memory available in MBs: 2781
time since start: ('seconds passed since start:', 34635)
current time is 34704.5793594
Memory available in MBs: 2328
time since start: ('seconds passed since start:', 34659)
Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_45 (LSTM)               (None, 7, 7)              392       
_________________________________________________________________
dropout_45 (Dropout)         (None, 7, 7)              0         
_________________________________________________________________
lstm_46 (LSTM)               (None, 7, 7)              420       
_________________________________________________________________
dropout_46 (Dropout)         (None, 7, 7)              0         
_________________________________________________________________
lstm_47 (LSTM)               (None, 7)                 420       
_________________

KeyboardInterrupt: 

In [ ]:
for each_sector in sectors:
    pipeline_features(each_sector)
    psutil.virtual_memory()